# Segmentation

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
pip install visualise_spacy_tree

In [52]:
pip install PyPDF2

In [53]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy import displacy
import visualise_spacy_tree
#from IPython.display import image,display
import PyPDF2

from numpy import loadtxt
import numpy as np
import pandas as pd
import os
import fnmatch
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import stopwords,wordnet
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag,RegexpParser
import nltk

In [54]:
pip install pdfplumber

In [55]:
import pdfplumber
import regex as re

In [56]:
def process_segmentation():
    ch=""
    names=""
    start=0
    process_df=pd.DataFrame(columns=['Process Name','start','Corpus'])
    with pdfplumber.open("/content/drive/MyDrive/PMBOK 5th.pdf") as pdf:
        for i in range (338,344):
            page=pdf.pages[i]
            pages=page.extract_text()
            for line in pages.split('\n'):
                if re.match('\d{2}.\d\s[A-Za-z]+\s[A-Za-z]',line):
                    if start!=0:
                        process_df=process_df.append({'Process Name':name,'start':start,'Corpus':ch[1:]},ignore_index=True)
                    ch=''
                    start=i
                    name=line[5:]
                ch=ch+'_ '+line
        process_df=process_df.append({'Process Name':name,'start':start,'Corpus':ch[1:]},ignore_index=True)
        return(process_df)

In [57]:
X=process_segmentation()

In [58]:
X

,Process Name,start,Corpus
0,Plan risk Management,338,11.1 Plan risk Management_ Plan Risk Manageme...


In [59]:
ch=""
with pdfplumber.open("/content/drive/MyDrive/PMBOK 5th.pdf")as temp:
    for i in range(338,344):
        first_page=temp.pages[i]
        ch=ch+first_page.extract_text()

In [60]:
ch

'11 - PROJECT RISK MANAGEMENT\n11.1 Plan risk Management\nPlan Risk Management is the process of defining how to conduct risk management activities for a project. The \nkey benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate \nwith both the risks and the importance of the project to the organization. The risk management plan is vital to \ncommunicate with and obtain agreement and support from all stakeholders to ensure the risk management process \nis supported and performed effectively over the project life cycle. The inputs, tools and techniques, and outputs of \nthis process are depicted in Figure 11-2. Figure 11-3 depicts the data flow diagram of the process.\nInputs Tools & Techniques Outputs\n.1 Project management plan .1 Analytical techniques .1 Risk management plan\n.2 Project charter .2 Expert judgment\n.3 Stakeholder register .3 Meetings\n.4 Enterprise environmental  \n  factors\n.5 Organizational process  \n  assets

In [61]:
inputs_titles=re.compile(r'\d+\..+: Inputs\n')
tools_and_techniques=re.compile(r'\d+\..+: tools and techniques\n')
outputs_titles=re.compile(r'\d+\..+: outputs\n')
get_subtitles=re.compile(r'[^|\n](\d+\.){2,4}\d+ [a-z|\s|A-Z]+(\d+)?\n')
title=re.compile(r'\d+ (.+): Inputs\n')

In [62]:
sections = inputs_titles.finditer(ch)
df = []
inputs = []
tt = []
outputss=[]
lst_sec = [elem for elem in sections]
for i in range(0,len(lst_sec)):
    
    if i == (len(lst_sec)-1):
        print(lst_sec[i].group())
        print("processus : ", title.search(lst_sec[i].group()).group(1))
        df.append(title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        str_between = ch[int(lst_sec[i].span()[1]):]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtitles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
        #####End liste des inputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtitles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"
        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
        ######End liste des tt
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtitles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
        ######End liste des outputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro

    else:
        
        print(lst_sec[i].group())
        # print("processus : ", title.search(lst_sec[i].group()).group(1))
        #Extraire toute la partie entre 2 processus
        df.append(title.search(lst_sec[i].group()).group(1))
        str_between = ch[int(lst_sec[i].span()[1]):int(lst_sec[i+1].span()[0])]
        t_and_t = tools_and_techniques.search(str_between)
        ch_inputs_techniques = str_between[:int(t_and_t.span()[0])]
        lst_sub_inputs = get_subtitles.finditer(ch_inputs_techniques)
        #######Liste des inputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_inputs]
        #print(lst2)
        l=list()
        nom_input = "p"+str(i+1)+"_inputs"
        l.append(nom_input)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        inputs.append(l)
        #####End liste des inputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_inputs]
        print(t_and_t.group())
        outputs = outputs_titles.search(str_between)
        ch_techniques_outputs = str_between[int(t_and_t.span()[1]):int(outputs.span()[0])]
        lst_sub_tt = get_subtitles.finditer(ch_techniques_outputs)
        ######Liste des tt
        lst2 = [a.group().split('\n')[0] for a in lst_sub_tt]
        #print(lst2)
        l=list()
        nom_tt = "p"+str(i+1)+"_tt"
        l.append(nom_tt)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        tt.append(l)
        ######End liste des tt
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_tt]
        print(outputs.group())
        ch_outputs = str_between[int(outputs.span()[1]):]
        lst_sub_outputs = get_subtitles.finditer(ch_outputs)
        ######Liste des outputs
        lst2 = [a.group().split('\n')[0] for a in lst_sub_outputs]
        #print(lst2)
        l=list()
        nom_output = "p"+str(i+1)+"_outputs"
        l.append(nom_output)
        for j in lst2 :
            a=j
            a=a.replace('\n','')
            a = re.sub(r'[*\d]', '', a)
            a=a.replace('.','')
            a = re.sub(r"^\s+","",a)
            l.append(a)
            print("\t"+j)
        outputss.append(l)
        ######End liste des outputs
        #[print("\t"+x.group().split('\n')[0]) for x in lst_sub_outputs if '-' not in x.gro
        print('-'*70)

11.1.1 Plan risk Management: Inputs

processus :  Plan risk Management
	11.1.1.1 Project Management Plan
	11.1.1.2 Project charter
	11.1.1.3 Stakeholder register
	11.1.1.4 Enterprise Environmental Factors
	11.1.1.5 organizational Process Assets
11.1.2 Plan risk Management: tools and techniques

	11.1.2.1 Analytical techniques
	11.1.2.2 Expert Judgment
	11.1.2.3 Meetings
11.1.3 Plan risk Management: outputs

	11.1.3.1 risk Management Plan


In [63]:
for i in inputs:
    a=i[0]
    exec('%s = %s' %(a, i[:]))
print(outputss)

[['p1_outputs', 'risk Management Plan']]


In [64]:
for i in outputss:
    a=i[0]
    exec('%s = %s' %(a, i[:]))
p1_inputs

['p1_inputs',
 'Project Management Plan',
 'Project charter',
 'Stakeholder register',
 'Enterprise Environmental Factors',
 'organizational Process Assets']

In [65]:
outputss

[['p1_outputs', 'risk Management Plan']]

In [66]:
for i in tt:
    a=i[0]
    exec('%s = %s' %(a, i[:]))
print(tt)


[['p1_tt', 'Analytical techniques', 'Expert Judgment', 'Meetings']]


In [67]:
tt

[['p1_tt', 'Analytical techniques', 'Expert Judgment', 'Meetings']]

In [68]:
inputs

[['p1_inputs',
  'Project Management Plan',
  'Project charter',
  'Stakeholder register',
  'Enterprise Environmental Factors',
  'organizational Process Assets']]

In [69]:
df

['Plan risk Management']

In [70]:
df_concept=pd.DataFrame()
df_concept['concept']=["Plan risk Management","Plan risk Management inputs","Plan risk Management outputs","Plan risk Management tools_and_techniques"]
df_concept

,concept
0,Plan risk Management
1,Plan risk Management inputs
2,Plan risk Management outputs
3,Plan risk Management tools_and_techniques


In [71]:
inputs[0][0]

'p1_inputs'

In [72]:

df_inputs=pd.DataFrame()
df_inputs["individual"]=inputs[0]
df_inputs

,individual
0,p1_inputs
1,Project Management Plan
2,Project charter
3,Stakeholder register
4,Enterprise Environmental Factors
5,organizational Process Assets


In [73]:

df_outputs=pd.DataFrame()
df_outputs["individual"]=outputss[0]
df_outputs

,individual
0,p1_outputs
1,risk Management Plan


In [74]:

df_tt=pd.DataFrame()
df_tt["individual"]=tt[0]
df_tt

,individual
0,p1_tt
1,Analytical techniques
2,Expert Judgment
3,Meetings


# Preprocessing

In [75]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [76]:
ch=ch.lower()
ch=ch.replace("\n"," ")
ch=ch.replace("\t"," ")
A=pd.DataFrame()
A['sentence']=sent_tokenize(ch)

In [77]:
A.style.set_properties(subset=['sentence'])

,sentence
0,11 - project risk management 11.1 plan risk management plan risk management is the process of defining how to conduct risk management activities for a project.
1,"the key benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate with both the risks and the importance of the project to the organization."
2,the risk management plan is vital to communicate with and obtain agreement and support from all stakeholders to ensure the risk management process is supported and performed effectively over the project life cycle.
3,"the inputs, tools and techniques, and outputs of this process are depicted in figure 11-2. figure 11-3 depicts the data flow diagram of the process."
4,"inputs tools & techniques outputs .1 project management plan .1 analytical techniques .1 risk management plan .2 project charter .2 expert judgment .3 stakeholder register .3 meetings .4 enterprise environmental factors .5 organizational process assets 11 figure 11-2. plan risk management: inputs, tools & techniques, and outputs project risk management 4.1 • project charter develop project charter 11.1 plan risk management 4.2 • enterprise develop project environmental management factors • risk plan • project • organizational management management process assets plan plan 11.4 11.2 perform 13.1 identify quantitative identify risks risk analysis stakeholders • stakeholder register 11.3 11.5 perform plan risk enterprise/ qualitative responses risk analysis organization figure 11-3. plan risk management data flow diagram ©2013 project management institute."
5,"a guide to the project management body of knowledge (pmbok® guide) – fifth edition 313 licensed to: jorge diego fuentes sanchez pmi memberid: 2399412 this copy is a pmi member benefit, not for distribution, sale, or reproduction.11 - project risk management careful and explicit planning enhances the probability of success for other risk management processes."
6,planning is also important to provide sufficient resources and time for risk management activities and to establish an agreed- upon basis for evaluating risks.
7,the plan risk management process should begin when a project is conceived and should be completed early during project planning.
8,"11.1.1 plan risk management: inputs 11.1.1.1 project management plan in planning risk management, all approved subsidiary management plans and baselines should be taken into consideration in order to make the risk management plan consistent with them."
9,the risk management plan is also a component of the project management plan.


In [78]:
pip install spacy download en_core_web_sm

In [79]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [80]:
A['filtered']=A['sentence'].apply(lambda s : " ".join([word.text for word in nlp(s) if not word.pos_ in ['ADV']]))
A["filtered"]=A["sentence"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
A['filtered']=A['sentence'].apply(lambda s : " ".join([word for word in s.split() if word !="the"]))

In [81]:
def removeHeader(text,word):
    new=text.replace(word,"")
    return new

In [82]:
import regex as re
def typo1(text):
    new=text.replace("˚","fl")
    return new
def typo2(text):
    new=text.replace("˜","fi")
    return new
def typo3(text):
    new=text.replace("1"," ")
    return new
def typo4(text):
    new=text.replace("."," ")
    return new
def Split(text):
    x=re.findall('.[^A-Z]*', text)
    x = ' '.join(x) 
    return x


In [83]:

A['filtered']=A['filtered'].apply(lambda s : "".join(removeHeader(s,"A Guide to Project Management Body of Knowledge (PMBOK ® Guide) Œ Fifth Edition 11")))
A['filtered']=A['filtered'].apply(lambda s : "".join(removeHeader(s,"- PROJECT RISK MANAGEMENT11")))
A['filtered']=A['filtered'].apply(lambda s : "".join(removeHeader(s,"- PROJECT RISK MANAGEMENTTable")))
A['filtered']=A['filtered'].apply(lambda s : "".join(removeHeader(s,"- PROJECT RISK MANAGEMENT")))
A['filtered']=A['filtered'].apply(lambda s : "".join(typo2(s)))
A['filtered']=A['filtered'].apply(lambda s : "".join(typo3(s)))
A['filtered']=A['filtered'].apply(lambda s : "".join(typo4(s)))
A['filtered']=A['filtered'].apply(lambda s : "".join(Split(s)))


In [84]:
def syns(s):
    l=[]
    for w in nlp(s):
        syn=wordnet.synsets(w.text)
        a=w.text
        if len(syn)>0 and w.pos_ in ['NOUN','VERB']:
            a=syn[0].name().split('.')[0]
        l.append(a)
    return l
def stopwords(s):
    doc=nlp(s)
    tokens = [token.text for token in doc]
    filtered = [token.text for token in doc if token.is_stop == False]
    return filtered
def postag(s):
    doc=nlp(s)
    pos = [[token.text,token.pos_] for token in doc]
    return pos
lemmatizer=WordNetLemmatizer()
def lemm(s):
    l=[]
    for word,tag in pos_tag(word_tokenize(s)):
        wntag=tag[0].lower()
        wntag=wntag if wntag in ['a','r','n','v'] else None
        lemma=lemmatizer.lemmatize(word,wntag) if wntag else word
        l.append(lemma)
    return l
def extractVerbNoun(s):
    doc=nlp(s)
    sent=[]
    for token in doc:
        
        if(token.pos_=='AUX') or (token.pos_=='VERB') :
            phrase=' '
            for sub_tok in token.lefts:
                if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    phrase+=sub_tok.text
                    phrase+=' '+token.text
                    for sub_tok in token.rights:
                        if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                            phrase+=' '+sub_tok.text
                            sent.append(phrase)
                            
    return sent

In [85]:
#A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.is_stop != False]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
A["filtered"]=A["filtered"].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))

In [86]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
A['filtered']=A['filtered'].apply(lambda s : " ".join(lemm(s)))
A['syn']=A['filtered'].apply(lambda s : "  ".join(syns(s)))
tokens=[word_tokenize(sent) for sent in A.filtered]
A['tokens']=tokens
A['pos_tag']=A["filtered"].apply(lambda s: postag(s))
A['pattern']=A["filtered"].apply(lambda s: extractVerbNoun(s))
A.style.set_properties(subset=['sentence'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,sentence,filtered,syn,tokens,pos_tag,pattern
0,11 - project risk management 11.1 plan risk management plan risk management is the process of defining how to conduct risk management activities for a project.,project risk management plan risk management plan risk management be process define conduct risk management activity project,undertaking hazard management plan hazard management plan hazard management be procedure specify behavior hazard management activity undertaking,"['project', 'risk', 'management', 'plan', 'risk', 'management', 'plan', 'risk', 'management', 'be', 'process', 'define', 'conduct', 'risk', 'management', 'activity', 'project']","[['project', 'NOUN'], ['risk', 'NOUN'], ['management', 'NOUN'], ['plan', 'NOUN'], ['risk', 'NOUN'], ['management', 'NOUN'], ['plan', 'NOUN'], ['risk', 'NOUN'], ['management', 'NOUN'], ['be', 'AUX'], ['process', 'NOUN'], ['define', 'NOUN'], ['conduct', 'NOUN'], ['risk', 'NOUN'], ['management', 'NOUN'], ['activity', 'NOUN'], ['project', 'NOUN']]",[' management be define']
1,"the key benefit of this process is it ensures that the degree, type, and visibility of risk management are commensurate with both the risks and the importance of the project to the organization.",benefit process be it ensure degree type visibility risk management be risk importance project organization,benefit procedure be it guarantee degree type visibility hazard management be hazard importance undertaking organization,"['benefit', 'process', 'be', 'it', 'ensure', 'degree', 'type', 'visibility', 'risk', 'management', 'be', 'risk', 'importance', 'project', 'organization']","[['benefit', 'NOUN'], ['process', 'NOUN'], ['be', 'AUX'], ['it', 'PRON'], ['ensure', 'VERB'], ['degree', 'NOUN'], ['type', 'NOUN'], ['visibility', 'NOUN'], ['risk', 'NOUN'], ['management', 'NOUN'], ['be', 'AUX'], ['risk', 'NOUN'], ['importance', 'NOUN'], ['project', 'NOUN'], ['organization', 'NOUN']]",[' management be organization']
2,the risk management plan is vital to communicate with and obtain agreement and support from all stakeholders to ensure the risk management process is supported and performed effectively over the project life cycle.,risk management plan be communicate obtain agreement support stakeholder ensure risk management process be support performed project life cycle,hazard management plan be communicate obtain agreement support stakeholder guarantee hazard management procedure be support perform undertaking life cycle,"['risk', 'management', 'plan', 'be', 'communicate', 'obtain', 'agreement', 'support', 'stakeholder', 'ensure', 'risk', 'management', 'process', 'be', 'support', 'performed', 'project', 'life', 'cycle']","[['risk', 'NOUN'], ['management', 'NOUN'], ['plan', 'NOUN'], ['be', 'AUX'], ['communicate', 'VERB'], ['obtain', 'VERB'], ['agreement', 'NOUN'], ['support', 'NOUN'], ['stakeholder', 'NOUN'], ['ensure', 'VERB'], ['risk', 'NOUN'], ['management', 'NOUN'], ['process', 'NOUN'], ['be', 'AUX'], ['support', 'NOUN'], ['performed', 'VERB'], ['project', 'NOUN'], ['life', 'NOUN'], ['cycle', 'NOUN']]",[' process be cycle']
3,"the inputs, tools and techniques, and outputs of this process are depicted in figure 11-2. figure 11-3 depicts the data flow diagram of the process.",tool technique output process be depict figure -2 figure depict data flow diagram process,tool technique end_product procedure be depict figure -2 figure picture data flow diagram procedure,"['tool', 'technique', 'output', 'process', 'be', 'depict', 'figure', '-2', 'figure', 'depict', 'data', 'flow', 'diagram', 'process']","[['tool', 'NOUN'], ['technique', 'NOUN'], ['output', 'NOUN'], ['process', 'NOUN'], ['be', 'AUX'], ['depict', 'ADJ'], ['figure', 'NOUN'], ['-2', 'PROPN'], ['figure', 'NOUN'], ['depict', 'NOUN'], ['data', 'NOUN'], ['flow', 'NOUN'], ['diagram', 'PROPN'], ['process', 'NOUN']]",[' process be process']
4,"inputs tools & techniques outputs .1 project management plan .1 analytical techniques .1 risk management plan .2 project charter .2 exp

In [87]:
from spacy import displacy
from collections import Counter
import en_core_web_sm

def entity(text):
    doc = nlp(text)
    print([(X.text, X.label_) for X in doc.ents])
A['entity']=A["filtered"].apply(lambda s: entity(s))

[]
[]
[]
[]
[('diagram © project management institute', 'ORG')]
[('pmbok guide fifth edition', 'ORG'), ('jorge diego', 'PERSON'), ('pmi memberid', 'PERSON')]
[]
[]
[]
[]
[]
[]
[]
[]
[('© project management institute', 'ORG')]
[('pmbok guide fifth edition', 'ORG'), ('jorge diego', 'PERSON'), ('pmi memberid', 'PERSON')]
[]
[]
[]
[]
[]
[]
[('© project management institute', 'ORG')]
[('pmbok guide fifth edition', 'ORG'), ('jorge diego', 'PERSON'), ('pmi memberid', 'PERSON')]
[]
[]
[]
[('contingency reserve', 'ORG')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('3', 'CARDINAL'), ('© project management institute', 'ORG')]
[('pmbok guide fifth edition', 'ORG'), ('jorge diego', 'PERSON'), ('pmi memberid', 'PERSON')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('© project management institute', 'ORG')]
[('pmbok guide fifth edition', 'ORG'), ('jorge diego', 'PERSON'), ('pmi memberid', 'PERSON'), ('> %', 'PERCENT'), ('> %', 'PERCENT')]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[('3', 'CARDINAL')

# Annotation extraction

In [88]:
def Split(text):
    x=re.findall('.[^A-Z]*', text)
    x = ' '.join(x) 
    return x
def punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s
def syns(s):
    l=[]
    for w in nlp(s):
        syn=wordnet.synsets(w.text)
        a=w.text
        if len(syn)>0 and w.pos_ in ['NOUN','VERB']:
            a=syn[0].name().split('.')[0]
        l.append(a)
    return l
def stopwords(s):
    doc=nlp(s)
    tokens = [token.text for token in doc]
    filtered = [token.text for token in doc if token.is_stop == False]
    return filtered
def postag(s):
    doc=nlp(s)
    pos = [[token.text,token.pos_] for token in doc]
    return pos
lemmatizer=WordNetLemmatizer()
def lemm(s):
    l=[]
    for word,tag in pos_tag(word_tokenize(s)):
        wntag=tag[0].lower()
        wntag=wntag if wntag in ['a','r','n','v'] else None
        lemma=lemmatizer.lemmatize(word,wntag) if wntag else word
        l.append(lemma)
    return l
def entity(text):
    doc=nlp(text)
    return [e for e in doc.ents]

def similarity(text1,text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)
def extractVerbNoun(s):
    doc=nlp(s)
    sent=[]
    for token in doc:
        
        if(token.pos_=='AUX') or (token.pos_=='VERB') :
            phrase=' '
            for sub_tok in token.lefts:
                if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                    phrase+=sub_tok.text
                    phrase+=' '+token.text
                    for sub_tok in token.rights:
                        if(sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                            phrase+=' '+sub_tok.text
                            sent.append(phrase)
                            
    return sent

In [89]:
input_df=pd.DataFrame()
input_df["individual"]=inputs[0]
input_df = input_df.drop(input_df.index[0])
p_m_p=['The project management plan provides baseline or current state of risk-affected areas including scope schedule and cost','The project charter can provide various inputs such as high-level risks, high-level project descriptions and high-level requirements','The stakeholder register, which contains all details related to the project’s stakeholders provides an overview of their roles','The enterprise environmental factors that can influence the Plan Risk Management process include but are not limited to risk attitudes thresholds and tolerances that describe the degree of risk that an organization will withstand',"Risk categories and Common definitions of concepts and terms and Risk statement formats and Standard templates and Roles and responsibilities and Authority levels for decision making and Lessons learned"]
input_df['annotation']=p_m_p

input_df=input_df.reset_index(drop=True)
input_df

,individual,annotation
0,Project Management Plan,The project management plan provides baseline ...
1,Project charter,The project charter can provide various inputs...
2,Stakeholder register,"The stakeholder register, which contains all d..."
3,Enterprise Environmental Factors,The enterprise environmental factors that can ...
4,organizational Process Assets,Risk categories and Common definitions of conc...


In [90]:
output_df=pd.DataFrame()
output_df["individual"]=outputss[0]
output_df = output_df.drop(output_df.index[0])
output_df=output_df.reset_index(drop=True)
p_m_p=["The risk management plan is a component of the project management plan and describes how risk management activities will be structured and performed"]
output_df['annotation']=p_m_p
output_df

,individual,annotation
0,risk Management Plan,The risk management plan is a component of the...


In [91]:
tools_and_techniques_df=pd.DataFrame()
tools_and_techniques_df["individual"]=tt[0]
tools_and_techniques_df = tools_and_techniques_df.drop(tools_and_techniques_df.index[0])
tools_and_techniques_df=tools_and_techniques_df.reset_index(drop=True)
p_m_p=['Analytical techniques are used to understand and define the overall risk management context of the project','Senior management and Project stakeholders and Project managers who have worked on projects in the same area and Subject matter experts (SMEs) in business or project area and Industry groups and consultants, and Professional and technical associations.','Project teams hold planning meetings to develop the risk management plan']
tools_and_techniques_df['annotation']=p_m_p
tools_and_techniques_df

,individual,annotation
0,Analytical techniques,Analytical techniques are used to understand a...
1,Expert Judgment,Senior management and Project stakeholders and...
2,Meetings,Project teams hold planning meetings to develo...


In [92]:
concept_df=pd.DataFrame()
concept = ['Plan risk Management']
concept_df["concept"]=concept
p_m_p=['Plan Risk Management is the process of defining how to conduct risk management activities for a project']
concept_df['annotation']=p_m_p
concept_df

,concept,annotation
0,Plan risk Management,Plan Risk Management is the process of definin...


In [93]:
input_df['annotation']=input_df['annotation'].apply(lambda s : "".join(punct(s)))
input_df['annotation']=input_df['annotation'].apply(lambda s : "".join(Split(s)))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
input_df['annotation']=input_df['annotation'].apply(lambda s : " ".join(lemm(s)))
input_df['pattern']=input_df["annotation"].apply(lambda s: extractVerbNoun(s))
input_df.style.set_properties(subset=['annotation'])

,individual,annotation,pattern
0,Project Management Plan,project management plan provide state riskaffected area include scope schedule cost,[' plan provide area']
1,Project charter,project charter can provide inputs risk project description requirement,[' charter provide requirement']
2,Stakeholder register,stakeholder register contain detail project stakeholder provide overview role,[' stakeholder provide role']
3,Enterprise Environmental Factors,enterprise factor can influence Plan Risk Management process include be limit to risk attitude threshold tolerance describe degree risk organization will withstand,[]
4,organizational Process Assets,Risk category definition concept term Risk statement format Standard templates Roles responsibility Authority level decision making Lessons learn,[' Standard templates decision']


In [94]:
input_df["pattern"][3]=["enterprice influence process"]
input_df["pattern"][4]=["assets influence plan "]

In [95]:
output_df['annotation']=output_df['annotation'].apply(lambda s : "".join(punct(s)))
output_df['annotation']=output_df['annotation'].apply(lambda s : "".join(Split(s)))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
output_df['annotation']=output_df['annotation'].apply(lambda s : " ".join(lemm(s)))
output_df['pattern']=output_df["annotation"].apply(lambda s: extractVerbNoun(s))
output_df.style.set_properties(subset=['annotation'])

,individual,annotation,pattern
0,risk Management Plan,risk management plan be component project management plan describe risk management activity will be structure performed,[' plan describe activity']


In [96]:
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : "".join(punct(s)))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : "".join(Split(s)))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
tools_and_techniques_df['annotation']=tools_and_techniques_df['annotation'].apply(lambda s : " ".join(lemm(s)))
tools_and_techniques_df['pattern']=tools_and_techniques_df["annotation"].apply(lambda s: extractVerbNoun(s))
tools_and_techniques_df.style.set_properties(subset=['annotation'])

,individual,annotation,pattern
0,Analytical techniques,technique be use understand define risk management context project,[]
1,Expert Judgment,management Project stakeholder Project manager who have work project area matter expert S M Es business project area Industry group consultant association,[' who have association']
2,Meetings,Project team hold planning meeting develop risk management plan,[' meeting develop plan']


In [97]:
tools_and_techniques_df['pattern'][0]=["technique define context"]
tools_and_techniques_df['pattern'][2]=["team plan plan"]

In [98]:
concept_df['annotation']=concept_df['annotation'].apply(lambda s : "".join(punct(s)))
concept_df['annotation']=concept_df['annotation'].apply(lambda s : "".join(Split(s)))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["NUM"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PUNCT"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["PART"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADP"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["DET"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["CCONJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["SCONJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADJ"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s:" ".join([word.text for word in nlp(s) if not word.pos_ in ["ADV"]]))
concept_df['annotation']=concept_df['annotation'].apply(lambda s : " ".join(lemm(s)))
concept_df['pattern']=concept_df["annotation"].apply(lambda s: extractVerbNoun(s))
concept_df.style.set_properties(subset=['annotation'])

,concept,annotation,pattern
0,Plan risk Management,Plan Risk Management be process define conduct risk management activity project,[' Management be define']


# Ontology

In [217]:
pip install rdflib

In [218]:
pip install owlready2

In [219]:
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer

In [220]:
g = Graph()
g.bind("owl",OWL)
g.bind("pr","http://example.org/projet#")
ns_url = "http://example.org/projet#"
g.add((URIRef('http://example.org/projet'),RDF.type, OWL.Ontology ))

<Graph identifier=N7e7a257cc3e3464a9928fc901166c974 (<class 'rdflib.graph.Graph'>)>

In [221]:
#adding classes and sub classes
for c in df_concept["concept"]:
    cl = URIRef(ns_url+c.replace(" ","_"))
    g.add((cl,RDF.type, OWL.Class))
    for i in range(len(X)):
        if X.loc[i,'Process Name'] != c:
            clp = URIRef(ns_url+X.loc[i, 'Process Name'].replace(" ","_"))
            g.add((cl,RDFS.subClassOf, clp))

In [222]:
g

<Graph identifier=N7e7a257cc3e3464a9928fc901166c974 (<class 'rdflib.graph.Graph'>)>

In [223]:
import pprint
for stmt in g:
    pprint.pprint(stmt)

(rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_inputs'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management'))
(rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_inputs'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.URIRef('http://example.org/projet'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Ontology'))
(rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_tools_and_techniques'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'),
 rdflib.term.URIR

In [224]:
df

['Plan risk Management']

In [225]:
df_concept

,concept
0,Plan risk Management
1,Plan risk Management inputs
2,Plan risk Management outputs
3,Plan risk Management tools_and_techniques


In [226]:
domains1=["has inputs","has outputs","has tools and techniques"]
obj1=["Plan risk Management","Plan risk Management", "Plan risk Management"]
range1=["Plan risk Management inputs","Plan risk Management outputs","Plan risk Management tools and techniques"]

In [227]:
#add object proprietes
df_objprop_domaine=pd.DataFrame()
df_objprop_domaine["object property"]=domains1
df_objprop_domaine["Domaine"]=obj1
df_objprop_domaine["Range"]=range1
df_objprop_domaine

,object property,Domaine,Range
0,has inputs,Plan risk Management,Plan risk Management inputs
1,has outputs,Plan risk Management,Plan risk Management outputs
2,has tools and techniques,Plan risk Management,Plan risk Management tools and techniques


In [228]:
for i in range(len(df_objprop_domaine)):
    c = URIRef(ns_url+df_objprop_domaine.loc[i,'object property'].replace(" ","_"))
    domaine = URIRef(ns_url+df_objprop_domaine.loc[i,'Domaine'].replace(" ","_"))
    rang = URIRef(ns_url+df_objprop_domaine.loc[i,'Range'].replace(" ","_"))
    g.add((c,RDF.type,OWL.ObjectProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,rang))

In [229]:
import pprint
for stmt in g:
    pprint.pprint(stmt)

(rdflib.term.URIRef('http://example.org/projet#has_tools_and_techniques'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'))
(rdflib.term.URIRef('http://example.org/projet#has_outputs'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management'))
(rdflib.term.URIRef('http://example.org/projet#has_tools_and_techniques'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_tools_and_techniques'))
(rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))
(rdflib.term.URIRef('http://example.org/projet'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('htt

In [230]:
#Add individuals
df_individual=pd.DataFrame()
df_individual['concept']=["Plan risk Management inputs","Plan risk Management inputs","Plan risk Management inputs","Plan risk Management inputs","Plan risk Management inputs"]
df_individual['annotation']=input_df["annotation"]
df_individual['individual']=input_df["individual"]
df_individual

,concept,annotation,individual
0,Plan risk Management inputs,project management plan provide state riskaffe...,Project Management Plan
1,Plan risk Management inputs,project charter can provide inputs risk projec...,Project charter
2,Plan risk Management inputs,stakeholder register contain detail project st...,Stakeholder register
3,Plan risk Management inputs,enterprise factor can influence Plan Risk Mana...,Enterprise Environmental Factors
4,Plan risk Management inputs,Risk category definition concept term Risk sta...,organizational Process Assets


In [231]:
for i in range (len(df_individual)):
    c = URIRef(ns_url+df_individual.loc[i,'concept'].replace(" ","_"))
    ind = URIRef(ns_url+df_individual.loc[i,'individual'].replace(" ","_"))
    g.add((ind,RDF.type,c))

In [232]:
df_individual=pd.DataFrame()
df_individual['concept']=["Plan risk Management outputs"]
df_individual['annotation']=output_df["annotation"]
df_individual['individual']=output_df["individual"]
df_individual

,concept,annotation,individual
0,Plan risk Management outputs,risk management plan be component project mana...,risk Management Plan


In [233]:
for i in range (len(df_individual)):
    c = URIRef(ns_url+df_individual.loc[i,'concept'].replace(" ","_"))
    ind = URIRef(ns_url+df_individual.loc[i,'individual'].replace(" ","_"))
    g.add((ind,RDF.type,c))

In [234]:
df_individual=pd.DataFrame()
df_individual['concept']=["Plan risk Management tools and techniques","Plan risk Management tools and techniques","Plan risk Management tools and techniques"]
df_individual['annotation']=tools_and_techniques_df["annotation"]
df_individual['individual']=tools_and_techniques_df["individual"]
df_individual

,concept,annotation,individual
0,Plan risk Management tools and techniques,technique be use understand define risk manage...,Analytical techniques
1,Plan risk Management tools and techniques,management Project stakeholder Project manager...,Expert Judgment
2,Plan risk Management tools and techniques,Project team hold planning meeting develop ris...,Meetings


In [235]:
for i in range (len(df_individual)):
    c = URIRef(ns_url+df_individual.loc[i,'concept'].replace(" ","_"))
    ind = URIRef(ns_url+df_individual.loc[i,'individual'].replace(" ","_"))
    g.add((ind,RDF.type,c))

In [236]:
import pprint
for stmt in g:
    pprint.pprint(stmt)

(rdflib.term.URIRef('http://example.org/projet#has_tools_and_techniques'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'))
(rdflib.term.URIRef('http://example.org/projet#organizational_Process_Assets'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_inputs'))
(rdflib.term.URIRef('http://example.org/projet#has_outputs'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management'))
(rdflib.term.URIRef('http://example.org/projet#has_tools_and_techniques'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'),
 rdflib.term.URIRef('http://example.org/projet#Plan_risk_Management_tools_and_techniques'))
(rdflib.term.URIRef('http://example.org/projet#Meetings'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax

In [237]:
L1=[]
L2=[]
L3=[]

for i in range(5):
  L1.append(input_df["pattern"][i][0].split()[0])
  L2.append(input_df["pattern"][i][0].split()[1])
  L3.append(input_df["pattern"][i][0].split()[2])

input_df["subject"]=L1
input_df["property"]=L2
input_df["object"]=L3

input_df

,individual,annotation,pattern,subject,property,object
0,Project Management Plan,project management plan provide state riskaffe...,[ plan provide area],plan,provide,area
1,Project charter,project charter can provide inputs risk projec...,[ charter provide requirement],charter,provide,requirement
2,Stakeholder register,stakeholder register contain detail project st...,[ stakeholder provide role],stakeholder,provide,role
3,Enterprise Environmental Factors,enterprise factor can influence Plan Risk Mana...,[enterprice influence process],enterprice,influence,process
4,organizational Process Assets,Risk category definition concept term Risk sta...,[assets influence plan ],assets,influence,plan


In [238]:
L1=[]
L2=[]
L3=[]

for i in range(1):
  L1.append(output_df["pattern"][i][0].split()[0])
  L2.append(output_df["pattern"][i][0].split()[1])
  L3.append(output_df["pattern"][i][0].split()[2])

output_df["subject"]=L1
output_df["property"]=L2
output_df["object"]=L3


output_df

,individual,annotation,pattern,subject,property,object
0,risk Management Plan,risk management plan be component project mana...,[ plan describe activity],plan,describe,activity


In [239]:
L1=[]
L2=[]
L3=[]
for i in range(3):
  L1.append(tools_and_techniques_df["pattern"][i][0].split()[0])
  L2.append(tools_and_techniques_df["pattern"][i][0].split()[1])
  L3.append(tools_and_techniques_df["pattern"][i][0].split()[2])
tools_and_techniques_df["subject"]=L1
tools_and_techniques_df["property"]=L2
tools_and_techniques_df["object"]=L3

tools_and_techniques_df

,individual,annotation,pattern,subject,property,object
0,Analytical techniques,technique be use understand define risk manage...,[technique define context],technique,define,context
1,Expert Judgment,management Project stakeholder Project manager...,[ who have association],who,have,association
2,Meetings,Project team hold planning meeting develop ris...,[team plan plan],team,plan,plan


In [240]:
L1=[]
L2=[]
L3=[]

for i in range(1):
  L1.append(concept_df["pattern"][i][0].split()[0])
  L2.append(concept_df["pattern"][i][0].split()[1])
  L3.append(concept_df["pattern"][i][0].split()[2])

concept_df["subject"]=L1
concept_df["property"]=L2
concept_df["object"]=L3

concept_df

,concept,annotation,pattern,subject,property,object
0,Plan risk Management,Plan Risk Management be process define conduct...,[ Management be define],Management,be,define


In [241]:
df_objprop_domaine=pd.DataFrame()
df_objprop_domaine=concept_df
for i in range(len(df_objprop_domaine)):
    c = URIRef(ns_url+df_objprop_domaine.loc[i,'property'].replace(" ","_"))
    domaine = URIRef(ns_url+df_objprop_domaine.loc[i,'concept'].replace(" ","_"))
    rang = URIRef(ns_url+df_objprop_domaine.loc[i,'object'].replace(" ","_"))
    g.add((c,RDF.type,OWL.ObjectProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,rang))
df_objprop_domaine=output_df
for i in range(len(df_objprop_domaine)):
    c = URIRef(ns_url+df_objprop_domaine.loc[i,'property'].replace(" ","_"))
    domaine = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
    rang = URIRef(ns_url+df_objprop_domaine.loc[i,'object'].replace(" ","_"))
    g.add((c,RDF.type,OWL.ObjectProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,rang))
df_objprop_domaine=tools_and_techniques_df
for i in range(len(df_objprop_domaine)):
    c = URIRef(ns_url+df_objprop_domaine.loc[i,'property'].replace(" ","_"))
    domaine = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
    rang = URIRef(ns_url+df_objprop_domaine.loc[i,'object'].replace(" ","_"))
    g.add((c,RDF.type,OWL.ObjectProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,rang))
df_objprop_domaine=input_df
for i in range(len(df_objprop_domaine)):
    c = URIRef(ns_url+df_objprop_domaine.loc[i,'property'].replace(" ","_"))
    domaine = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
    rang = URIRef(ns_url+df_objprop_domaine.loc[i,'object'].replace(" ","_"))
    g.add((c,RDF.type,OWL.ObjectProperty))
    g.add((c,RDFS.domain,domaine))
    g.add((c,RDFS.range,rang))

In [242]:
df_objprop_domaine=pd.DataFrame()
df_objprop_domaine=concept_df
for i in range(len(df_objprop_domaine)):
   annotation = URIRef(ns_url+df_objprop_domaine.loc[i,'annotation'].replace(" ","_"))
   ind = URIRef(ns_url+df_objprop_domaine.loc[i,'concept'].replace(" ","_"))
   g.add((ind,RDFS.comment,annotation))

df_objprop_domaine=output_df
for i in range(len(df_objprop_domaine)):
   annotation = URIRef(ns_url+df_objprop_domaine.loc[i,'annotation'].replace(" ","_"))
   ind = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
   g.add((ind,RDFS.comment,annotation))

df_objprop_domaine=tools_and_techniques_df
for i in range(len(df_objprop_domaine)):
   annotation = URIRef(ns_url+df_objprop_domaine.loc[i,'annotation'].replace(" ","_"))
   ind = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
   g.add((ind,RDFS.comment,annotation))

df_objprop_domaine=input_df
for i in range(len(df_objprop_domaine)):
   annotation = URIRef(ns_url+df_objprop_domaine.loc[i,'annotation'].replace(" ","_"))
   ind = URIRef(ns_url+df_objprop_domaine.loc[i,'individual'].replace(" ","_"))
   g.add((ind,RDFS.comment,annotation))



In [243]:
g.serialize(destination="Processus1.owl")

<Graph identifier=N7e7a257cc3e3464a9928fc901166c974 (<class 'rdflib.graph.Graph'>)>